In [ ]:
%config Completer.use_jedi = False

In [ ]:
from fastai.vision.all import *

In [ ]:
train_df = pd.read_csv('../input/digit-recognizer/train.csv')
train_df.head(3)

In [ ]:
train_df = train_df.astype(np.uint8)

## Dataloader

In [ ]:
def get_img(line): return line.drop('label').values.reshape(28, 28)

In [ ]:
plt.imshow(get_img(train_df.loc[100]), cmap = 'gray');

In [ ]:
dblock = DataBlock(
    blocks = [ImageBlock(cls = PILImageBW), CategoryBlock],
    splitter = RandomSplitter(valid_pct = 0.2, seed = 42),
    get_x = get_img,
    get_y = ColReader('label')
)
dls = dblock.dataloaders(train_df, num_workers = 4, bs = 256)
dls.show_batch()

## Learner

In [ ]:
learn = cnn_learner(dls, resnet34, metrics = accuracy)
learn.fine_tune(10)

## Verification

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

## Inference

In [ ]:
test_df = pd.read_csv('../input/digit-recognizer/test.csv')
test_df.head(3)

In [ ]:
test_df['label'] = 99
test_df = test_df.astype(np.uint8)

In [ ]:
dl_test = dls.test_dl(test_df)

In [ ]:
preds, _ = learn.get_preds(dl = dl_test)

In [ ]:
predicted_labels = preds.argmax(dim = 1)
predicted_labels

In [ ]:
submission = pd.DataFrame({
    'ImageId': range(1, len(predicted_labels) + 1),
    'Label': predicted_labels
})
submission.to_csv('submission.csv', index = False)
submission.head()